In [1]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

In [14]:
%%sql
drop table if exists FiscalYearTable1;
CREATE TABLE FiscalYearTable1 (
  fiscal_year    INTEGER,
  start_date     DATE,
  end_date       DATE
);

--正常データ
INSERT INTO FiscalYearTable1 VALUES(1995, '1994-10-01', '1995-09-30');
INSERT INTO FiscalYearTable1 VALUES(1997, '1996-10-01', '1997-09-30');
INSERT INTO FiscalYearTable1 VALUES(1998, '1997-10-01', '1998-09-30');

--エラーデータ
INSERT INTO FiscalYearTable1 VALUES(1996, '1995-10-01', '1996-08-30');  -- 終了日が8月
INSERT INTO FiscalYearTable1 VALUES(1999, '1998-10-02', '1999-09-30');  -- 開始日が2日

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [12]:
%%sql
(SELECT f1.fiscal_year
FROM FiscalYearTable1 AS f1
WHERE '1995-06-30' BETWEEN f1.start_date AND f1.end_date);

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


fiscal_year
1995


## 実装

FiscalYearTable1は、各会計年度がいつから始まりいつで終わるのか、その日付の範囲を整理する。

特定の日付から会計期に該当する年を取得するクエリは👆の通り。

このテーブルにEXTRACT関数を使いながら、間違ったデータをinsertできないようにするにはどうすればよい？

## テーブルに課すべき制約

- fiscal_year は 主キー
- start_date, end_dateはNOT_NULL
- start_date, end_dateはunique
- end_date - start_dateは365日？